# 多级索引的取值与切片
- 对MultiIndex的取值和切片操作很直观，你可以直接把索引看成额外增加的维度。我们先来介绍Series多级索引的取值与切片方法，再介绍DataFrame的用法。

### Series多级索引

In [24]:
import pandas as pd
import numpy as np
index = [('California', 2000), ('California', 2010),
                ('New York', 2000), ('New York', 2010),
                ('Texas', 2000), ('Texas', 2010)]
index=pd.MultiIndex.from_tuples(index)
print(index)
print()
populations = [33871648, 37253956,
                      18976457, 19378102,
                      20851820, 25145561]
pop = pd.Series(populations,index=index)
print(pop)
print()
populations=np.array(populations).reshape(3,2)
data = pd.DataFrame(index=["California","New York","Texas"],columns=[2000,2010],data=populations)
print(data)
print()
pop=pop.reindex(index)
pop.index.names = ['state', 'year']
print(pop)

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

                2000      2010
California  33871648  37253956
New York    18976457  19378102
Texas       20851820  25145561

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64


In [20]:
# 可以通过对多个级别索引值获取单个元素
pop["California",2000]

33871648

In [21]:
# MultiIndex也支持局部取值（partial indexing），即只取索引的某一个层级
pop["California"]
# 获取的结果是一个新的Series

year
2000    33871648
2010    37253956
dtype: int64

In [22]:
# 还有局部切片，不过要求MultiIndex是按顺序排列的
pop.loc["California":"New York"]

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [23]:
# 如果索引已经排序，那么可以用较低层级的索引取值，第一层级的索引可以用空切片：

pop[:,2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [25]:
# 通过布尔掩码选择数据
pop[pop>22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [26]:
# 通过花哨的索引选择数据
pop[['California','Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Dataframe多级索引
- DataFrame多级索引的用法与Series类似

In [28]:
# 之前的体检报告的例子
import pandas as pd
import numpy as np
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])
data=np.arange(24).reshape(4,6)
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject    Bob      Guido      Sue     
type        HR Temp    HR Temp  HR Temp
year visit                             
2013 1       0    1     2    3   4    5
     2       6    7     8    9  10   11
2014 1      12   13    14   15  16   17
     2      18   19    20   21  22   23

- 由于DataFrame的基本索引是列索引，因此Series中多级索引的用法到了DataFrame中就应用在列上了。例如，可以通过简单的操作获取Guido的心率数据

In [29]:
health_data["Guido","HR"]

year  visit
2013  1         2
      2         8
2014  1        14
      2        20
Name: (Guido, HR), dtype: int32

In [30]:
# 与单索引类似，在上几节介绍的loc、iloc和ix索引器都可以使用
health_data.iloc[:2,:2]

subject    Bob     
type        HR Temp
year visit         
2013 1       0    1
     2       6    7

In [32]:
# 虽然这些索引器将多维数据当作二维数据处理，但是在loc和iloc中可以传递多个层级的索引元组
health_data.loc[:,('Bob','HR')]

year  visit
2013  1         0
      2         6
2014  1        12
      2        18
Name: (Bob, HR), dtype: int32

In [33]:
# 用Pandas的IndexSlice对象切片
idx=pd.IndexSlice
health_data.loc[idx[:,1],idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,0,2,4
2014,1,12,14,16


## 多级索引行列转换

- 使用多级索引的关键是掌握有效数据转换的方法。Pandas提供了许多操作，可以让数据在内容保持不变的同时，按照需要进行行列转换。之前我们用一个简短的例子演示过stack()和unstack()的用法，但其实还有许多合理控制层级行列索引的方法

#### 1.有序的索引和无序的索引

- 在前面的内容里，我们曾经简单提过多级索引排序，这里需要详细介绍一下。如果MultiIndex不是有序的索引，那么大多数切片操作都会失败

In [34]:
# 首先创建一个不安字典顺序排列的多级索引Series
import pandas as pd
import numpy as np
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.237893
      2      0.420309
c     1      0.898627
      2      0.151936
b     1      0.833357
      2      0.970307
dtype: float64

In [35]:
# 如果想对索引使用局部切片，那么错误就会出现：
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


- 局部切片和许多其他相似的操作都要求MultiIndex的各级索引是有序的（即按照字典顺序由A至Z）。为此，Pandas提供了许多便捷的操作完成排序，如sort_index()和sortlevel()方法。我们用最简单的sort_index()方法来演示：

In [36]:
data = data.sort_index()
data
# 索引排序之后，局部切片就正常使用了

char  int
a     1      0.237893
      2      0.420309
b     1      0.833357
      2      0.970307
c     1      0.898627
      2      0.151936
dtype: float64

In [37]:
data["a":"b"]

char  int
a     1      0.237893
      2      0.420309
b     1      0.833357
      2      0.970307
dtype: float64

#### 2. 索引stack与unstack
- 我们可以将一个多级索引数据集转换成简单的二维形式，
  可以通过level参数设置转换的索引层级：

In [40]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [38]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [39]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [41]:
# unstack()是stack()的逆操作，同时使用这两种方法让数据保持不变：
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 索引的设置与重置

- 层级数据维度转换的另一种方法是行列标签转换，可以通过reset_index方法实现。如果在上面的人口数据Series中使用该方法，则会生成一个列标签中包含之前行索引标签的state和year的Dataframe.也可以用数据的name属性为列设置名称。

In [42]:
pop_flat=pop.reset_index(name="population")
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [43]:
pop_flat.set_index(["state","year"])
# 在实践中，我发现用这种重建索引的方法处理数据集非常好用。

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## 多级索引的数据累计方法


In [47]:
import pandas as pd
import numpy as np
rad=np.random.seed(0)
columns = pd.MultiIndex.from_product([["Bob","Guido","Sue"],["HR","Temp"]],
                                  names=["subject",'type'])
index = pd.MultiIndex.from_product([[2013,2014],[1,2]],
                                  names=["year","visit"])
data=(np.random.randint(250,650,size=(4,6))) /10
health_data = pd.DataFrame(index=index,columns=columns,data=data)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      42.2  29.7  36.7  44.2  57.3  50.1
     2      44.5  60.9  25.9  46.1  52.7  49.2
2014 1      54.2  33.7  32.0  33.8  64.6  56.4
     2      44.3  28.9  33.7  42.4  33.8  58.7

In [48]:
# 计算每一年各项的平均值，可以将参数level设置为索引year():
data_mean=health_data.mean(level='year')
data_mean

subject    Bob        Guido          Sue       
type        HR  Temp     HR   Temp    HR   Temp
year                                           
2013     43.35  45.3  31.30  45.15  55.0  49.65
2014     49.25  31.3  32.85  38.10  49.2  57.55

In [50]:
# 如果再设置axis参数，就可以对列索引进行类似的累计操作了：
data_mean.mean(axis=1,level='type')

#通过这两行数据，我们就可以获取每一年所有人的平均心率和体温了。
#这种语法其实就是GroupBy功能的快捷方式，我们将在3.9节详细介绍。
#尽管这只是一个简单的示例，但是其原理和实际工作中遇到的情况类似。

type,HR,Temp
year,,
2013,43.216667,46.700000
2014,43.766667,42.316667


### Panel数据

- 这里还有一些Pandas的基本数据结构没有介绍到，包括pd.Panel对象和pd.Panel4D对象。这两种数据结构可以分别看成是（一维数组）Series和（二维数组）DataFrame的三维与四维形式。如果你熟悉Series和DataFrame的使用方法，那么Panel和Panel4D使用起来也会很简单，ix、loc和iloc索引器（详情请参见3.3节）在高维数据结构上的用法更是完全相同。
  Panel采用密集数据存储形式，而多级索引采用稀疏数据存储形式。在解决许多真实的数据集时，随着维度的不断增加，密集数据存储形式的效率将越来越低。但是这类数据结构对一些有特殊需求的应用还是有用的。

# 合并数据集：Concat与Append操作

- 将不同的数据源进行合并是数据科学中最有趣的事情之一，这既包括将两个不同的数据集非常简单地拼接在一起，也包括用数据库那样的连接（join）与合并（merge）操作处理有重叠字段的数据集。Series与DataFrame都具备这类操作，Pandas的函数与方法让数据合并变得快速简单。
- 先来用pd.concat函数演示一个Series与DataFrame的简单合并操作。之后，我们将介绍Pandas中更复杂的merge和join内存数据合并操作。

In [51]:
import pandas as pd
import numpy as np
def make_df(cols, ind):
    """一个简单的DataFrame"""
    data = {c: [str(c) + str(i) for i in ind]
           for c in cols}
    return pd.DataFrame(data, ind)
make_df("ABC",range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


### 知识回顾：NumPy数组的合并
- 合并Series与DataFrame与合并NumPy数组基本相同，后者通过2.2节中介绍的np.concatenate函数即可完成。你可以用这个函数将两个或两个以上的数组合并成一个数组

In [54]:
import numpy as np
import pandas as pd
x = [1,2,3]
y = [4,5,6]
z = [7,8,9]
np.concatenate([x,y,z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [61]:
p = np.random.seed(20)
x=np.random.randint(0,9,size=(3,3))
print(x)
print(np.concatenate([x,x],axis=1))

[[3 4 6]
 [7 2 0]
 [6 8 5]]
[[3 4 6 3 4 6]
 [7 2 0 7 2 0]
 [6 8 5 6 8 5]]


### 通过pd.concat实现简易合并

- Pandas有一个pd.concat()函数与np.concatenate语法类似，但是配置参数更多，功能也更强大

In [62]:
import pandas as pd
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [63]:
# 同样可以合并高维数组
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1)
print(df2)
print(pd.concat([df1,df2]))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


默认情况下，DataFrame的合并都是逐行进行的（默认设置是axis=0）。与np.concatenate()一样，pd.concat也可以设置合并坐标轴

In [66]:
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3)
print(df4)
print(pd.concat([df3, df4], axis=1))

    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


#### 索引重复
* np.concatenate与pd.concat最主要的差异之一就是Pandas在合并时会保留索引，即使索引是重复的！

In [68]:
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index  # 复制索引
print(x)
print(y)
print(pd.concat([x,y]))

# (1).捕捉索引重复的错误。如果你想要检测pd.concat()合并的结果中是否出现了重复的索引，
# 可以设置verify_integrity参数。将参数设置为True，合并时若有索引重复就会触发异常

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


In [69]:
#（2）忽略索引。有时索引无关紧要，那么合并时就可以忽略它们，可以通过设置ignore_index参数来实现。如果将参数设置为True，那么合并时将会创建一个新的整数索引。

print(x)
print(y)
print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [70]:
# (3)增加多级索引。另一种处理索引重复的方法是通过keys参数为数据源设置多级索引标签，这样结果数据就会带上多级索引：
print(x)
print(y)
print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


### 类似join的合并
- 前面介绍的简单示例都有一个共同特点，那就是合并的DataFrame都是同样的列名。而在实际工作中，需要合并的数据往往带有不同的列名，而pd.concat提供了一些选项来解决这类合并问题。看下面两个DataFrame，它们的列名部分相同，却又不完全相同：

In [73]:
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5)
print(df6)
print(pd.concat([df5, df6]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


默认情况下，某个位置上缺失的数据会用NaN表示。如果不想这样，可以用join和join_axes参数设置合并方式。默认的合并方式是对所有输入列进行并集合并（join='outer'），当然也可以用join='inner'实现对输入列的交集合并

In [74]:
import pandas as pd
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5)
print(df6)
print(pd.concat([df5, df6],join="inner")) # 交集合并

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


In [75]:
import pandas as pd
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5)
print(df6)
print(pd.concat([df5, df6],join="outer")) # 并集合并

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


另一种合并方式是直接确定结果使用的列名，设置join_axes参数，里面是索引对象构成的列表（是列表的列表）。如下面示例所示，将结果的列名设置为第一个输入的列名：

In [76]:
import pandas as pd
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5)
print(df6)
print(pd.concat([df5, df6],join_axes=[df5.columns]))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C
1   A1  B1  C1
2   A2  B2  C2
3  NaN  B3  C3
4  NaN  B4  C4


### append()方法
- 因为直接进行数组合并的需求非常普遍，所以Series和DataFrame对象都支持append方法，让你通过最少的代码实现合并功能。例如，你可以使用df1.append(df2)，效果与pd.concat([df1, df2])一样：        

In [77]:
print(df1); print(df2); print(df1.append(df2))

    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


* 需要注意的是，与Python列表中的append()和extend()方法不同，Pandas的append()不直接更新原有对象的值，而是为合并后的数据创建一个新对象。因此，它不能被称之为一个非常高效的解决方案，因为每次合并都需要重新创建索引和数据缓存。总之，如果你需要进行多个append操作，还是建议先创建一个DataFrame列表，然后用concat()函数一次性解决所有合并任务。